<a href="https://colab.research.google.com/github/Ivory-T/transformers/blob/master/KaggleCompetitionClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c cs98xspotifyclassification
!ls /content

Saving kaggle.json to kaggle.json
  0% 0.00/8.37k [00:00<?, ?B/s]
100% 8.37k/8.37k [00:00<00:00, 3.24MB/s]
  0% 0.00/38.2k [00:00<?, ?B/s]
100% 38.2k/38.2k [00:00<00:00, 40.2MB/s]
CS98XClassificationTest.csv   kaggle.json
CS98XClassificationTrain.csv  sample_data


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# load training & test datasets
train = pd.read_csv("/content/CS98XClassificationTrain.csv")
#train = train.append(train, sort =False)
test = pd.read_csv("/content/CS98XClassificationTest.csv")

In [ ]:
train["mainGenre"]=train["top genre"]

train.head()
train = train.dropna()

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876 entries, 0 to 452
Data columns (total 16 columns):
Id           876 non-null int64
title        876 non-null object
artist       876 non-null object
year         876 non-null int64
bpm          876 non-null int64
nrgy         876 non-null int64
dnce         876 non-null int64
dB           876 non-null int64
live         876 non-null int64
val          876 non-null int64
dur          876 non-null int64
acous        876 non-null int64
spch         876 non-null int64
pop          876 non-null int64
top genre    876 non-null object
mainGenre    876 non-null object
dtypes: int64(12), object(4)
memory usage: 116.3+ KB


In [ ]:
map = {"rock" : 1, "pop" : 2, "soul" : 3, "hip hop" : 4, "dance" : 5, 
              "blues" : 6, "adult standards" : 7, "folk" : 8, "disco" : 9}

for i in train["top genre"]:  
  if "pop" in i or "boy band" in i or "australian" in i or "latin" in i or "r&b" in i:
    train["mainGenre"].replace(i,map["pop"], inplace=True)
  elif i in ["afrobeat", "big room", "bubble trance", "bubblegum dance", "eurodance", "uk garage"] or "house" in i:
    train["mainGenre"].replace(i,map["dance"], inplace=True)
  elif "rock"in i or "metal" in i or  "boogaloo" in i or "brit" in i or "doo-wop" in i or "glam" in i or "merseybeat" in i or "permanent wave" in i:
    train["mainGenre"].replace(i,map["rock"], inplace=True)
  elif "soul" in i :
    train["mainGenre"].replace(i,map["soul"], inplace=True)
  elif "hip hop" in i or "rap" in i or "g funk" in i:
    train["mainGenre"].replace(i,map["hip hop"], inplace=True)
  elif "blues" in i or "jazz" in i or "bebop" in i:
    train["mainGenre"].replace(i,map["blues"], inplace=True)
  #elif "metal" in i:
    #train["mainGenre"].replace(i,map["metal"], inplace=True)
  elif "adult standards" in i or "chanson" in i or "classic girl" in i or "mellow" in i:
    train["mainGenre"].replace(i,map["adult standards"], inplace=True)
  elif "folk" in i or "belgian dance" in i or "country" in i or "german dance" in i or "yodeling" in i:
    train["mainGenre"].replace(i,map["folk"], inplace=True)
  #elif "jazz" in i or "bebop" in i:
    #train["mainGenre"].replace(i,map["jazz"], inplace=True)
  elif "disco" in i or "hi-nrg" in i:
    train["mainGenre"].replace(i,map["disco"], inplace=True)
  else:
    pass

train["mainGenre"] = pd.to_numeric(train["mainGenre"])
map["dance"]


5

In [ ]:
gg = train.groupby("mainGenre").count()

In [ ]:
#y = train["mainGenre"]
y = train["top genre"]
X = train.drop(columns=["mainGenre", "Id", "top genre", "title", "year", "artist", "pop"])
X

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch
0,107,31,45,-8,13,28,150,75,3
2,105,36,63,-9,13,67,245,11,3
3,170,28,47,-16,13,33,232,25,3
4,121,47,56,-8,15,40,193,45,3
5,110,56,71,-7,12,23,223,15,6
...,...,...,...,...,...,...,...,...,...
448,80,22,18,-17,10,16,214,92,4
449,148,81,53,-13,23,96,147,50,3
450,168,55,73,-8,20,61,289,23,14
451,165,87,64,-5,6,88,191,5,8


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(return_indices=True)
X_trainRUS, y_trainRUS, id_rus = rus.fit_sample(X_train, y_train)

X_trainRUS=pd.DataFrame(X_trainRUS)


from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_trainROS, y_trainROS = ros.fit_sample(X_train, y_train)

X_trainROS=pd.DataFrame(X_trainROS)

from imblearn.under_sampling import TomekLinks

tl = TomekLinks(return_indices=True, ratio='majority')
X_trainTL, y_trainTL, id_tl = tl.fit_sample(X_train, y_train)

X_trainTL=pd.DataFrame(X_trainTL)


from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=42)
X_trainCC, y_trainCC = cc.fit_sample(X_train, y_train)

X_trainCC=pd.DataFrame(X_trainCC)


"""
from imblearn.over_sampling import SMOTE

#smote = SMOTE(ratio='minority')
sm = SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=48, m_neighbors='deprecated', out_step='deprecated', kind='deprecated', svm_estimator='deprecated', n_jobs=1, ratio=None) 
#features_res, labels_res = sm.fit_sample(x_val, y_val)
X_sm, y_sm = sm.fit_sample(X, y)

Xsm=pd.DataFrame(X_sm)
ysm = y_sm
"""
#from imblearn.combine import SMOTETomek

#smt = SMOTETomek(ratio='auto')
#X_smt, y_smt = smt.fit_sample(X, y)

#Xsmt=pd.DataFrame(X_smt)
#ysmt = y_smt

#Xus = X
#yus = y
#yus=pd.DataFrame(y_tl)



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

"\nfrom imblearn.over_sampling import SMOTE\n\n#smote = SMOTE(ratio='minority')\nsm = SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=48, m_neighbors='deprecated', out_step='deprecated', kind='deprecated', svm_estimator='deprecated', n_jobs=1, ratio=None) \n#features_res, labels_res = sm.fit_sample(x_val, y_val)\nX_sm, y_sm = sm.fit_sample(X, y)\n\nXsm=pd.DataFrame(X_sm)\nysm = y_sm\n"

In [ ]:
#x_test = test.drop(columns=["Id"])
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
gnb = GaussianNB()


#Randome Under Sampling
y_pred = gnb.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSgnb = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = gnb.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSgnb = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = gnb.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLgnb = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = gnb.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCgnb = y_false/y_size*100

#cluster Centroids undersampling
y_pred = gnb.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percgnb = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))



In [ ]:
from sklearn.linear_model import SGDClassifier
#X_train, X_test, y_train, y_test = train_test_split(Xus, yus, test_size=0.2, random_state=0)

clf = SGDClassifier(loss="squared_epsilon_insensitive", penalty="l1", max_iter=5)
#clf.fit(X_train, y_train)
SGDClassifier(max_iter=5)

#Randome Under Sampling
y_pred = clf.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSclf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = clf.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSclf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = clf.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLclf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = clf.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCclf = y_false/y_size*100

#cluster Centroids undersampling
y_pred = clf.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percclf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_mo

In [ ]:
##random forest
from sklearn.model_selection import train_test_split
# implementing train-test-split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
#rfc.fit(X_train,y_train)
# predictions
#rfc_predict = rfc.predict(X_test)

#Randome Under Sampling
y_pred = rfc.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSrfc = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = rfc.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSrfc= y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = rfc.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLrfc = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = rfc.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCrfc = y_false/y_size*100

#cluster Centroids undersampling
y_pred = rfc.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percrfc = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))


#print("percentage false mislabeled: %f " %(y_percF))

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
#rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring=’roc_auc’)

In [ ]:
##suport vector machines linear
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
#svclassifier.fit(X_train, y_train)
#y_pred = svclassifier.predict(X_test)
#print("----------------------------------------------------------------------------")
#print("Linear Kernel")
#print("----------------------------------------------------------------------------")
#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
#Randome Under Sampling
y_pred = svclassifier.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSsvl = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = svclassifier.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSsvl = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = svclassifier.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLsvl = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = svclassifier.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCsvl = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))
y_pred = svclassifier.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percsvl = y_false/y_size*100

#print("----------------------------------------------------------------------------")
#print("Smote over- undersampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))


In [ ]:
##kernel svm (non straight line separable data)

##polynomial Kernel

from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(X_train, y_train)
#y_pred = svclassifier.predict(X_test)

#print("----------------------------------------------------------------------------")
#print("Poly Kernel")
#print("----------------------------------------------------------------------------")
#Randome Under Sampling
y_pred = svclassifier.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSsvp = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling

y_pred = svclassifier.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSsvp = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = svclassifier.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLsvp = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = svclassifier.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCsvp = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))
y_pred = svclassifier.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percsvp = y_false/y_size*100


##gaussian kernel
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)
#y_pred = svclassifier.predict(X_test)
#print("----------------------------------------------------------------------------")
#print("RBF Kernel")
#print("----------------------------------------------------------------------------")
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))
#Randome Under Sampling
y_pred = svclassifier.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSrbf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = svclassifier.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSrbf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = svclassifier.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLrbf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = svclassifier.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percrbf = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))
y_pred = svclassifier.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percrbf = y_false/y_size*100

##sigmoid kernel
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(X_train, y_train)
#print("----------------------------------------------------------------------------")
#print("Sigmoid Kernel")
#print("----------------------------------------------------------------------------")
#y_pred = svclassifier.predict(X_test)

#Randome Under Sampling
y_pred = svclassifier.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSsig = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = svclassifier.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSsig = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = svclassifier.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLsig = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = svclassifier.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCsig = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))
y_pred = svclassifier.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percsig = y_false/y_size*100


In [ ]:
##knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=9, metric='euclidean')
#knn.fit(X_train, y_train)
#y_pred = knn.predict(X_test)
#print("----------------------------------------------------------------------------")
#print("KNN")
#print("----------------------------------------------------------------------------")
#confusion_matrix(y_test, y_pred)
#Randome Under Sampling
y_pred = knn.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSknn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = knn.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSknn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = knn.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLknn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = knn.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCknn = y_false/y_size*100

y_pred = knn.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percknn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
model = VotingClassifier(estimators=[('rf', BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=50,
    max_samples=50, bootstrap=True, n_jobs=-1, random_state=42)), ('lr', LogisticRegression())])


y_pred = model.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSmod = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = model.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSmod = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = model.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLmod = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = model.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCmod = y_false/y_size*100

y_pred = model.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percmod = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
from sklearn.neural_network import MLPClassifier

nn= MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=200, activation= 'relu', hidden_layer_sizes=(10,30,10), random_state=1, shuffle=True)


y_pred = nn.fit(X_trainRUS, y_trainRUS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percRUSnn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Under Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Random over Sampling
y_pred = nn.fit(X_trainROS, y_trainROS).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percROSnn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Random Over Sampling")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#Tomek Links (Undersampling)
y_pred = nn.fit(X_trainTL, y_trainTL).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percTLnn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Tomek Links(Undersampling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))

#cluster Centroids undersampling
y_pred = nn.fit(X_trainCC, y_trainCC).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percCCnn = y_false/y_size*100

y_pred = nn.fit(X_train, y_train).predict(X_test)
y_false = (y_test != y_pred).sum()
y_size = X_test.shape[0]
y_percnn = y_false/y_size*100
#print("----------------------------------------------------------------------------")
#print("Clusetr Centroids (Undersapling)")
#print("----------------------------------------------------------------------------")
#print("Number of mislabeled points out of a total %d points : %d" %(y_size, y_false))
#print("percentage false mislabeled: %f " %(y_percF))





/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

In [ ]:
from astropy.table import QTable, Table, Column
t = Table()
t['gnb'] = [y_percRUSgnb, y_percROSgnb, y_percTLgnb, y_percCCgnb, y_percgnb]
t['clf'] = [y_percRUSclf, y_percROSclf, y_percTLclf, y_percCCclf, y_percclf]
t['rfc'] = [y_percRUSrfc, y_percROSrfc, y_percTLrfc, y_percCCrfc, y_percrfc]
t['svlin'] = [y_percRUSsvl, y_percROSsvl, y_percTLsvl, y_percCCsvl, y_percsvl]
t['svpol'] = [y_percRUSsvp, y_percROSsvp, y_percTLsvp, y_percCCsvp, y_percsvp]
t['svsig'] = [y_percRUSsig, y_percROSsig, y_percTLsig, y_percCCsig, y_percsig]
t['svpol'] = [y_percRUSrbf, y_percROSrbf, y_percTLrbf, y_percCCrbf, y_percrbf]
t['knn'] = [y_percRUSknn, y_percROSknn, y_percTLknn, y_percCCknn, y_percknn]
t['mod'] = [y_percRUSmod, y_percROSmod, y_percTLmod, y_percCCmod, y_percmod]
t['nn'] = [y_percRUSnn, y_percROSnn, y_percTLnn, y_percCCnn, y_percnn]
"""
t1 = Table()
t1['gnb'] = [y_percROSgnb, y_percSMTgnb]
t1['clf'] = [y_percROSclf, y_percSMTclf]
t1['rfc'] = [y_percROSrfc, y_percSMTrfc]
t1['svlin'] = [y_percROSsvl, y_percSMTsvl]
t1['svpol'] = [y_percROSsvp, y_percSMTsvp]
t1['svsig'] = [y_percROSsig, y_percSMTsig]
t1['svpol'] = [y_percROSrbf, y_percSMTrbf]
t1['knn'] = [y_percROSknn, y_percSMTknn]
t1['vo'] = [y_percROSvo, y_percSMTvo]
t1['nn'] = [y_percROSnn, y_percSMTnn]
"""
t



gnb,clf,rfc,svlin,svpol,svsig,knn,mod,nn
float64,float64,float64,float64,float64,float64,float64,float64,float64
92.04545454545455,97.72727272727273,96.5909090909091,92.04545454545455,92.04545454545455,100.0,93.18181818181817,94.31818181818183,94.31818181818183
87.5,98.86363636363636,88.63636363636364,88.63636363636364,94.31818181818183,100.0,93.18181818181817,95.45454545454545,100.0
82.95454545454545,88.63636363636364,81.81818181818183,86.36363636363636,85.22727272727273,88.63636363636364,80.68181818181817,81.81818181818183,84.0909090909091
95.45454545454545,100.0,98.86363636363636,95.45454545454545,100.0,100.0,90.9090909090909,96.5909090909091,100.0
95.45454545454545,100.0,77.27272727272727,80.68181818181817,85.22727272727273,89.77272727272727,79.54545454545455,72.72727272727273,81.81818181818183


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    StandardScaler(),
)

model = VotingClassifier(estimators=[('rf', BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=50,
    max_samples=50, bootstrap=True, n_jobs=-1, random_state=42)), ('lr', LogisticRegression())])

x_train = pipeline.fit_transform(X_train)
model = LogisticRegression(max_iter=1000).fit(x_train, y_train)
x_test = pipeline.transform(X_test)
pred = model.predict(x_test)
print(accuracy_score(pred, y_test))


0.26136363636363635


In [ ]:
#!kaggle competitions submit -c cs98xspotifyclassification -f submission.csv -m "This is a sample submisssion"